# Stock Data Analysis

This notebook analyzes WIG20 stock prices, comparing raw data with smoothed moving averages using different window sizes.


In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data import load_prices, smooth_prices

%matplotlib inline
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (14, 7)
plt.rcParams["font.size"] = 12

## 1. Load Data

Selected WIG20 companies for analysis: Allegro, CD Projekt, PKO BP.


In [ ]:
tickers = ['ALE.WA', 'CDR.WA', 'PKO.WA']
prices_df = load_prices(tickers, start="2023-01-01")
prices_df.head()

## 2. Smoothing

Comparison of raw closing prices with moving averages using 7-day and 30-day windows.


In [ ]:
window_sizes = [7, 30]
colors = ['#ff7f0e', '#2ca02c']

for ticker in tickers:
    plt.figure(figsize=(14, 6))
    
    raw_prices = prices_df[ticker]
    plt.plot(raw_prices.index, raw_prices.values, label=f'{ticker} - Raw', alpha=0.3, color='gray', linewidth=1)
    
    for window, color in zip(window_sizes, colors):
        smoothed = smooth_prices(raw_prices, window=window)
        plt.plot(smoothed.index, smoothed.values, label=f'MA ({window} days)', linewidth=2, color=color)
    
    plt.title(f'Price Analysis: {ticker}', fontsize=15, fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Price (PLN)')
    plt.legend()
    plt.tight_layout()
    plt.show()